In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [43]:
# m_from, m_to - skąd zabieramy, gdzie wsadzamy
# op - zabierana operacja, para (nr_maszyny, nr_operacji)
# pos - pozycja do wsadzenia op na maszynę m_to

# tutaj numery maszyn idą od 0
def lower_bound(G, m_from, op, m_to, pos):
    def alpha(k,i):
        assert i >= 0
        if k != m_from or solution[k].index(op) > i:
            return G.solution[k][i]
        #print k, i
        return G.solution[k][i+1]
    
    # jeśli longest_path nie istnieje, to powinien zwrócić None
    # te drogi poniżej są zawarte w H1 (pierwszej składowej G), więc
    # można by je liczyć trochę szybciej (tylko w H1)
    
    def R(k,opr):
        return G.longest_path_length((0,) + alpha(k,0), (0,) + opr, justH1=True)
    
    def Q(k,opr):
        last_op_k_idx = len(G.solution[k]) - 1
        if k == m_from:
            last_op_k_idx -= 1
        return G.longest_path_length((0,) + opr, (0,) + alpha(k, last_op_k_idx), justH1=True)

    def LB(l):
        op_old_weight = G.graph[(0,) + op].weight
        op_new_weight = G.problem[op[0]-1][op[1]-1][m_to+1]
        
        R1 = lambda: R(l, alpha(m_to, pos-1))
        def R2():
            r = R(l, op)
            return r - op_old_weight if r is not None else None
        Q1 = lambda: Q(l, alpha(m_to, pos))
        def Q2():
            q = Q(l, op)
            return q - op_old_weight if q is not None else None 
        
        # 0 w maxach po to, żeby mieć jakąś liczbę, gdy drogi nie istnieją
        if pos == 0:
            return max(R2(), 0) + max(Q1(), Q2(), 0) + op_new_weight
                                     
        len_m_to = len(G.solution[m_to])
        if m_to == m_from:
            len_m_to -= 1
        if pos == len_m_to:
            return max(R1(), R2(), 0) + max(Q2(), 0) + op_new_weight
        
        maxR = max(R1(), R2(), 0)
        maxQ = max(Q1(), Q2(), 0)
        
        return maxR + maxQ + op_new_weight # a może tu ma być op_new_weight?
    print [LB(l) for l in xrange(len(solution))]
    return max([LB(l) for l in xrange(len(solution))])

# JUŻ NIEAKTUALNE:
# co jeśli m_from == m_to (czy w ogóle może tak być?) - alpha(m_to, pos) może nie istnieć

In [44]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors
G.topological_sort()

In [58]:
G.longest_path_length((0,9,1), (0,1,1))

In [45]:
lower_bound(G, 0, (9,1), 0, 1)

[386, 571, 512, 772, 843, 765, 818, 839, 834, 831, 76]


843